In [ ]:
import redis
import os
from dotenv import load_dotenv
load_dotenv()

r = redis.Redis(
    host="redis-15814.c305.ap-south-1-1.ec2.redns.redis-cloud.com",
    port=15814,
    username="default",
    password=os.getenv("REDIS_PASSWORD"),
    decode_responses=True
)

print(r.ping())

True


In [2]:
r.set('foo', 'bar')
r.get('foo')

'bar'

In [16]:
# Set a key
r.set('name', 'Hemant')

# Get a key
print(r.get('name'))  # -> 'Hemant'

# Check if a key exists
print(r.exists('name'))  # -> 1 (True)

# Delete a key
r.delete('name')


Hemant
1


1

In [17]:
r.set('temp_key', 'temp', ex=5)  # expires in 5 sec
print(r.ttl('temp_key'))  # Time to live

5


In [18]:
r.set('count', 10)

# Increment by 1
r.incr('count')  # -> 11

# Increment by custom value
r.incrby('count', 5)  # -> 16

# Append to a string
r.set('msg', 'Hello')
r.append('msg', ' World')
print(r.get('msg'))  # -> 'Hello World'

Hello World


In [19]:
# Set hash fields
r.hset('user:1000', 'name', 'Hemant')
r.hset('user:1000', 'age', 30)

# Get a single field
print(r.hget('user:1000', 'name'))  # -> 'Hemant'

# Get all fields
print(r.hgetall('user:1000'))
# -> {'name': 'Hemant', 'age': '30'}

# Delete a field
r.hdel('user:1000', 'age')

# Check if field exists
print(r.hexists('user:1000', 'name'))  # -> True

Hemant
{'name': 'Hemant', 'age': '30'}
True


In [20]:
r.delete('tasks')  # clean slate

# Push to left (acts like stack)
r.lpush('tasks', 'task1', 'task2')

# Push to right (acts like queue)
r.rpush('tasks', 'task3')

# Pop from left
print(r.lpop('tasks'))  # -> 'task2'

# Pop from right
print(r.rpop('tasks'))  # -> 'task3'

# Get range of elements
r.lpush('tasks', 'task1', 'task2', 'task3')
print(r.lrange('tasks', 0, -1))


task2
task3
['task3', 'task2', 'task1', 'task1']


In [21]:
r.sadd('tags', 'python', 'redis', 'backend')

# Add duplicate — won't appear twice
r.sadd('tags', 'redis')

# Members
print(r.smembers('tags'))  # -> {'python', 'redis', 'backend'}

# Remove a value
r.srem('tags', 'backend')

# Check membership
print(r.sismember('tags', 'python'))  # True


{'backend', 'redis', 'python'}
1


In [3]:
r.hset('user-session:123', mapping={
    'name': 'John',
    "surname": 'Smith',
    "company": 'Redis',
    "age": 29
})

r.hgetall('user-session:123')


{'name': 'John', 'surname': 'Smith', 'company': 'Redis', 'age': '29'}

In [22]:
# Add members with scores
r.zadd('leaderboard', {'Alice': 100, 'Bob': 150, 'Eve': 120})

# Get members by rank
print(r.zrange('leaderboard', 0, -1, withscores=True))
# [('Alice', 100.0), ('Eve', 120.0), ('Bob', 150.0)]

# Get members by score range
print(r.zrangebyscore('leaderboard', 100, 130))

# Remove a member
r.zrem('leaderboard', 'Alice')


[('Alice', 100.0), ('Eve', 120.0), ('Bob', 150.0)]
['Alice', 'Eve']


1

In [24]:
pipe = r.pipeline()

pipe.set('foo', 'bar')
pipe.incr('counter')
pipe.get('foo')

result = pipe.execute()
print(result)


[True, 2, 'bar']


In [5]:
pipe = r.pipeline()
pipe.set('foo', 5)
pipe.set('bar', 18.5)
pipe.set('blee', "hello world!")
pipe.execute()

[True, True, True]

In [14]:
from redis.commands.search.field import TextField
from redis.commands.search.query import Query
from redis.commands.search.index_definition import IndexDefinition

# r.ft().create_index(
#     (TextField("name"), TextField("lastname")),
#     definition=IndexDefinition(prefix=["test:"]),
# )

# r.hset("test:1", "name", "James")
# r.hset("test:1", "lastname", "Brown")

# Query with default DIALECT 2
query = "@name: James Brown"
q = Query(query)
res = r.ft().search(q)
print(res)

# Query with explicit DIALECT 1
query = "@name: James Brown"
q = Query(query).dialect(1)
res = r.ft().search(q)
print(res)



Result{1 total, docs: [Document {'id': 'test:1', 'payload': None, 'name': 'James', 'lastname': 'Brown'}]}
Result{0 total, docs: []}


In [15]:
from sentence_transformers import SentenceTransformer
from redis.commands.search.query import Query
from redis.commands.search.field import TextField, TagField, VectorField
from redis.commands.search.index_definition import IndexDefinition, IndexType
from redis.commands.json.path import Path

import numpy as np
import redis

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

try:
    r.ft("vector_idx").dropindex(True)
except redis.exceptions.ResponseError:
    pass

schema = (
    TextField("content"),
    TagField("genre"),
    VectorField("embedding", "HNSW", {
        "TYPE": "FLOAT32",
        "DIM": 384,
        "DISTANCE_METRIC":"L2"
    })
)

r.ft("vector_idx").create_index(
    schema,
    definition=IndexDefinition(
        prefix=["doc:"], index_type=IndexType.HASH
    )
)

content = "That is a very happy person"

r.hset("doc:0", mapping={
    "content": content,
    "genre": "persons",
    "embedding": model.encode(content).astype(np.float32).tobytes(),
})

content = "That is a happy dog"

r.hset("doc:1", mapping={
    "content": content,
    "genre": "pets",
    "embedding": model.encode(content).astype(np.float32).tobytes(),
})

content = "Today is a sunny day"

r.hset("doc:2", mapping={
    "content": content,
    "genre": "weather",
    "embedding": model.encode(content).astype(np.float32).tobytes(),
})

q = Query(
    "*=>[KNN 3 @embedding $vec AS vector_distance]"
).return_field("score").dialect(2)

query_text = "That is a happy person"

res = r.ft("vector_idx").search(
    q, query_params={
        "vec": model.encode(query_text).astype(np.float32).tobytes()
    }
)

print(res)

Result{3 total, docs: [Document {'id': 'doc:0', 'payload': None}, Document {'id': 'doc:1', 'payload': None}, Document {'id': 'doc:2', 'payload': None}]}


In [ ]:
import redis
from redis.commands.json.path import Path
import redis.commands.search.aggregation as aggregations
import redis.commands.search.reducers as reducers
from redis.commands.search.field import TextField, NumericField, TagField
from redis.commands.search.index_definition import IndexDefinition, IndexType
from redis.commands.search.query import Query
import redis.exceptions


user1 = {
    "name": "Paul John",
    "email": "paul.john@example.com",
    "age": 42,
    "city": "London"
}

user2 = {
    "name": "Eden Zamir",
    "email": "eden.zamir@example.com",
    "age": 29,
    "city": "Tel Aviv"
}

user3 = {
    "name": "Paul Zamir",
    "email": "paul.zamir@example.com",
    "age": 35,
    "city": "Tel Aviv"
}

schema = (
    TextField("$.name", as_name="name"),
    TagField("$.city", as_name="city"),
    NumericField("$.age", as_name="age")
)

indexCreated = r.ft("idx:users").create_index(
    schema,
    definition=IndexDefinition(
        prefix=["user:"], index_type=IndexType.JSON
    )
)

user1Set = r.json().set("user:1", Path.root_path(), user1)
user2Set = r.json().set("user:2", Path.root_path(), user2)
user3Set = r.json().set("user:3", Path.root_path(), user3)

findPaulResult = r.ft("idx:users").search(
    Query("Paul @age:[30 40]")
)

print(findPaulResult)


citiesResult = r.ft("idx:users").search(
    Query("Paul").return_field("$.city", as_field="city")
).docs

print(citiesResult)


req = aggregations.AggregateRequest("*").group_by(
    '@city', reducers.count().alias('count')
)

aggResult = r.ft("idx:users").aggregate(req).rows
print(aggResult)

hashSchema = (
    TextField("name"),
    TagField("city"),
    NumericField("age")
)

hashIndexCreated = r.ft("hash-idx:users").create_index(
    hashSchema,
    definition=IndexDefinition(
        prefix=["huser:"], index_type=IndexType.HASH
    )
)

huser1Set = r.hset("huser:1", mapping=user1)
huser2Set = r.hset("huser:2", mapping=user2)
huser3Set = r.hset("huser:3", mapping=user3)

findPaulHashResult = r.ft("hash-idx:users").search(
    Query("Paul @age:[30 40]")
)

print(findPaulHashResult)

r.close()


Result{1 total, docs: [Document {'id': 'user:3', 'payload': None, 'json': '{"name":"Paul Zamir","email":"paul.zamir@example.com","age":35,"city":"Tel Aviv"}'}]}
[Document {'id': 'user:1', 'payload': None, 'city': 'London'}, Document {'id': 'user:3', 'payload': None, 'city': 'Tel Aviv'}]
[['city', 'London', 'count', '1'], ['city', 'Tel Aviv', 'count', '2']]
Result{1 total, docs: [Document {'id': 'huser:3', 'payload': None, 'name': 'Paul Zamir', 'email': 'paul.zamir@example.com', 'age': '35', 'city': 'Tel Aviv'}]}
